In [1]:
import pandas as pd
import numpy as np

In [6]:
df=pd.read_csv("data.csv")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5235 entries, 0 to 5234
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           5235 non-null   int64  
 1   State                                5235 non-null   object 
 2   AgencyType                           5235 non-null   object 
 3   AgencyName                           5235 non-null   object 
 4   Population                           5235 non-null   float64
 5   DrugNarcoticOffenses                 4752 non-null   float64
 6   DrugNarcoticViolations               4724 non-null   float64
 7   DrugEquipmentViolations              3550 non-null   float64
 8   GamblingOffenses                     219 non-null    float64
 9   BettingWagering                      75 non-null     float64
 10  OperatingPromotingAssistingGambling  132 non-null    float64
 11  GamblingEquipmentViolations   

In [10]:
null_percentages = df.isnull().sum() / len(df) * 100
null_percentages

Unnamed: 0                              0.000000
State                                   0.000000
AgencyType                              0.000000
AgencyName                              0.000000
Population                              0.000000
DrugNarcoticOffenses                    9.226361
DrugNarcoticViolations                  9.761223
DrugEquipmentViolations                32.187202
GamblingOffenses                       95.816619
BettingWagering                        98.567335
OperatingPromotingAssistingGambling    97.478510
GamblingEquipmentViolations            98.510029
SportsTampering                        99.942693
PornographyObsceneMaterial             69.436485
ProstitutionOffenses                   86.150907
Prostitution                           88.787011
AssistingorPromotingProstitution       93.333333
WeaponLawViolations                    29.531996
dtype: float64

In [15]:
df['DrugNarcoticOffenses'].describe()

count     4752.000000
mean       119.711490
std        386.257438
min          1.000000
25%         10.000000
50%         34.000000
75%         98.000000
max      14921.000000
Name: DrugNarcoticOffenses, dtype: float64

In [ ]:
def 

# Train Test Split

In [ ]:
X=df.drop('mh_disorder_current', axis=1)

In [ ]:
y=df['mh_disorder_current']

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=32)

NameError: name 'X' is not defined

# Scores Generator

In [4]:
def score_generator(model, name='not-specified', type='train'):
    y_pred = model.predict(x_train)

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)

    metrics_dict = {
        'Mean Absolute Error (MAE)': mae,
        'Mean Squared Error (MSE)': mse,
        'Root Mean Squared Error (RMSE)': rmse,
        'R-squared (R2)': r2
    }

    return metrics_dict


# Linear Model